In [4]:
import random
import copy

def replace(a_list, symb_a, symb_d):
    '''replaces ints with chosen symbols for dead and alive cells'''
    return [symb_a if x == 1 else symb_d for x in a_list]


def format_round(game):
    '''creates visualization for a round of game of life'''
    formatted_game = []
    
    for line in game:
        newline = replace(line, symb_alive, symb_dead)
        formatted_game.append("  ".join(str(x) for x in newline))
        
    print("\nROUND {} ------------>".format(round_count))
    print('\n'.join(map(str,formatted_game)))
    return game


def create_canvas(height, width):
    '''creates the canvas on which game of life will be played'''
    game = []
    
    for line in range(height):
        filled_line = []
        for position in range(width):
            filled_line.append(random.randint(0, 1))
        game.append(filled_line)
    
    return format_round(game)

In [5]:
def cor(pos):
    '''corrects position within a line if its out of range'''
    if pos < 0:
        return 0
    if pos > width:
        return width
    return pos


def check_line(line):
    '''returns IndexError if a line is -1. reason: it doesn't exist, \
    but python will evaluate the last line (line[-1])'''
    if line == -1:
        raise IndexError
        
    return line


def check_position(game, line, pos):
    '''returns count of life around a given position (line and pos) \
    within a specified game of life (game)'''
    
    surrounding = sum(game[line][cor(pos-1):cor(pos+2)]) - game[line][pos]
    
    if not line == 0:
        try:
            surrounding += sum(game[check_line(line-1)][cor(pos-1):cor(pos+2)])
        except IndexError:
            pass
        
    try:
        surrounding += sum(game[line+1][cor(pos-1):cor(pos+2)])
    except IndexError:
        pass
    
    return surrounding


def alive_or_dead(this_round, line, pos):
    '''evaluates whether a cell should be alive or dead in the next round'''
    life_around = check_position(this_round, line, pos)
    
    if life_around == 3:
        return 1
    
    if life_around == 2:
        return this_round[line][pos]
    
    return 0

In [12]:
import imageio
import time
import numpy as np

def scale_for_animation(frame, times_big):
    '''scales up single round so it can be visualized'''
    return np.kron(frame, np.ones((times_big,times_big)))

def create_gif(frames, fps):
    '''writes and saves gif of played game of life. uses imageio; ext can also be .mp4'''
    ext = ".gif"
    name = str(int(time.time())) + ext
    writer = imageio.get_writer(name, fps=fps)
    
    for frame in frames:
        writer.append_data(frame)
        
    writer.close()
    
    print("saved video as: {}".format(name))

In [10]:
def play_conway():
    '''generator that plays the game'''
    
    this_round = create_canvas(height, width)
    yield this_round
    
    while True:
        
        next_round = []
        
        for line in range(0, height):
            new_line = []
            
            for pos in range(0, width):
                new_pos = alive_or_dead(this_round, line, pos)
                new_line.append(new_pos)
               
            next_round.append(new_line)
        
        this_round = copy.deepcopy(next_round)
        yield next_round

In [13]:
# variables that can be adjusted:
rounds_to_play = 50
height = 20
width = 20
symb_alive = "O"
symb_dead = " "

# static variables:
game_rounds = play_conway()
round_count = 1
visual = []

#let's play:
for game_round in game_rounds:
    
    # needed for visual:
    if not round_count == 1:
        b = scale_for_animation(game_round, 40)
        visual.append(b)
    
    format_round(game_round)
    
    if round_count >= rounds_to_play:
        print("Finished game after {} rounds; bye.".format(round_count))
        create_gif(visual, 5)
        break
        
    round_count += 1


ROUND 1 ------------>
      O     O        O  O     O  O     O     O  O  O      
O  O  O  O              O  O  O        O     O     O  O  O
   O  O     O  O  O  O                 O     O  O        O
   O  O  O     O  O     O        O  O  O  O           O   
         O     O           O     O  O  O              O   
O  O           O  O  O                 O  O     O  O  O   
                     O  O  O  O  O  O        O     O  O   
   O     O     O  O        O           O     O     O  O  O
      O     O  O  O     O  O  O        O  O  O            
O  O              O  O  O        O     O     O  O     O   
                     O     O        O        O        O   
   O                       O     O        O  O  O  O     O
            O     O  O  O        O  O  O  O  O  O  O     O
      O     O     O  O  O  O                 O  O  O      
                  O        O     O           O        O  O
   O           O        O     O  O  O     O           O  O
O  O  O     O        O     O  O  


ROUND 11 ------------>
                                    O  O                  
                                 O     O  O               
                              O  O        O  O  O         
                           O     O  O     O  O            
                     O  O           O  O     O            
O  O              O              O     O  O     O         
O  O           O  O           O                           
               O  O     O                                 
            O           O           O  O     O  O     O   
         O  O                       O  O  O  O     O      
      O           O                          O            
         O                    O  O  O                     
                           O     O  O           O  O  O   
                        O        O        O  O  O  O  O   
O  O                 O                    O     O     O   
O  O                 O        O  O        O               
                              O 


ROUND 21 ------------>
                                                          
                                       O                  
                  O                 O     O               
               O        O           O     O               
            O           O              O                  
         O  O           O                                 
            O           O                                 
            O           O                                 
            O        O                       O  O  O      
            O  O  O                       O  O        O   
            O  O     O  O           O  O     O           O
                  O  O     O                    O  O  O   
                  O           O  O        O     O         
                  O           O  O     O  O           O   
O  O                 O     O  O  O                 O     O
O  O                 O  O     O  O                 O  O   
                           O  O 


ROUND 31 ------------>
                                                          
                                       O                  
                                    O     O               
                                    O                     
                                    O        O            
                                       O     O            
                                       O  O               
                                                          
                                    O  O           O  O   
               O  O                 O  O           O  O   
               O  O                    O        O         
                                          O        O      
                                          O     O         
                                          O               
O  O                                   O                 O
O  O                                   O                 O
                                


ROUND 41 ------------>
                           O  O  O                        
                           O                              
                              O                           
                                                          
                                    O                     
                                 O     O                  
                              O        O              O  O
                                 O              O         
                                    O  O  O  O  O  O     O
               O  O                    O  O              O
               O  O                          O  O         
                                                O         
                                                          
                                                          
O  O                                                      
O  O                                                      
                                


ROUND 50 ------------>
                           O  O                           
                           O  O                           
                                                          
                                                          
                              O  O                        
                              O  O                        
                                                          
                                       O                  
                                    O  O  O               
               O  O                 O     O  O            
               O  O                       O  O  O         
                                          O  O            
                                          O               
                                                          
O  O                                                      
O  O                                                      
                                

saved video as: 1544103508.gif
